In [51]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col



In [52]:
#Connection path to the java database connection jars for MongoDB
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "C:\\MongoDBSparkCOnnector\\mongo-spark-connector_2.12-3.0.2.jar,C:\\MongoDBSparkConnector\\mongo-java-driver-3.12.9.jar" pyspark-shell'

In [53]:
#set up spark session
spark = SparkSession.builder \
    .appName("MongoDB and PySpark") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/GoT.twitter") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/GoT.twitter") \
    .getOrCreate()

In [54]:
#Read in montly data csv files

jan = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Jan2016 GOT.csv',inferSchema=True, header =True)
feb = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Feb2016 GOT.csv',inferSchema=True, header =True)
march = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\March2016 GOT.csv',inferSchema=True, header =True)
april = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\April2016 GOT.csv',inferSchema=True, header =True)
may = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\May2016 GOT.csv',inferSchema=True, header =True)
june = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\June2016 GOT.csv',inferSchema=True, header =True)
july = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\July2016 GOT.csv',inferSchema=True, header =True)
august = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\August2016 GOT.csv',inferSchema=True, header =True)
september = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Sept2016 GOT.csv',inferSchema=True, header =True)
october = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Oct2016 GOT.csv',inferSchema=True, header =True)
november = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Nov2016 GOT.csv',inferSchema=True, header =True)
december = spark.read.option("multiLine", "true").option("escape", "\"").csv('C:\\data files\\Dec2016 GOT.csv',inferSchema=True, header =True)

In [55]:
#append into one dataframe
got_df = jan.union(feb).union(march).union(april).union(may).union(june).union(july).union(august).union(september).union(october).union(november).union(december)

In [56]:
got_df.count()

268051

In [57]:
got_df.printSchema()

root
 |-- poster: string (nullable = true)
 |-- recipient: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- tweet date: string (nullable = true)
 |-- tweet id: long (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- retweet status: boolean (nullable = true)



In [58]:
#Get rid of false positives
#Drop all where hashtag != gameofthrones

got_df = got_df.withColumn("hashtags", lower(col("hashtags")))
got_df = got_df.filter(got_df.hashtags.contains("gameofthrones"))


In [59]:
#Store in MongoDB
got_df.write.format("mongo").mode("append").save()



In [60]:
 spark.stop()